<div align="center">
  <h1><b> Quantum Information </b></h1>
  <h2> Quantum Channels </h2>
</div>

<br>
<b>Author:</b> <a target="_blank" href="https://github.com/camponogaraviera">Lucas Camponogara Viera</a>

# Table of Contents

- [Coherent error channel](#coherent)
- [Depolarizing error channel](#depolarizing) 
- [Amplitude damping channel](#amp)
- [Phase damping channel](#pdamp)
- [Phase-amplitude damping error channel](#phaseamplitude)
- [Kraus error channel](#kraus) 

# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/python" width="45" hspace="0px" vspace="0px"></a> Importing dependencies

In [ ]:
from math import exp, pi, sqrt

from qiskit.circuit.library import RXGate
from qiskit.providers.aer.noise import NoiseModel, ReadoutError, coherent_unitary_error, depolarizing_error

from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, \
                                                              phase_amplitude_damping_error, \
                                                              kraus_error

# [Coherent error channel](https://qiskit.org/ecosystem/aer/stubs/qiskit_aer.noise.coherent_unitary_error.html)<a name="coherent" /> 

**Example:** adding an $\epsilon$ over rotation to an RXGate on the first qubit:

In [3]:
# Parameters:
epsilon = pi/5 # Over rotation amount.
epsilon_rotation = RXGate(epsilon).to_matrix() # Get matrix representation.

# Create coherent error channel:
error = coherent_unitary_error(epsilon_rotation)

# Adding error to noise model on the rx gate for a single qubit:
noise_model = NoiseModel()
noise_model.add_quantum_error(error, ['rx'], qubits = [0])

Alternative, adding to a list of gates for all qubits:

In [4]:
# Adding error to noise model on a list of gates for all qubits:

instructions = ['id', 'x', 'y', 'z', 'rx', 'ry', 'rz'] # 'id' is the idle (identity) gate as placeholder.
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, instructions)

# [Depolarizing error channel](https://qiskit.org/documentation/stable/0.19/stubs/qiskit.providers.aer.noise.depolarizing_error.html#depolarizing-error)<a name="depolarizing" /> 

The depolarizing error is the simplest model of `incoherent noise` arising from `decoherence` or `imperfect control`. 

The depolarizing quantum error channel action on a density matrix (state) $\rho$ is defined as:

$$E(\rho) = (1-\lambda)\rho + \lambda Tr[\rho]\frac{I}{2^n}.$$

**Meaning:** transforms a pure state $\rho$ into a mixed state with some probability $\lambda$.

- Parameters:

    - $\frac{I}{2^n}$: is the a completely mixed state.
    - $\lambda$: is the depolarizing error parameter. Represents the probability of a bit-flip or phase-flip error occurring on each qubit in the circuit.
    - $n$: number of qubits for the error channel.

**Example:** adding a depolarizing error to an idle (identity) gate on the first qubit:

In [5]:
# Creating a depolarizing noise model with probability lambda_= 0.01 for a single qubit.

# Parameters:
lambda_ = 0.01 # Error parameter of 1%.
num_qubits = 1

# Create depolarizing error channel:
error = depolarizing_error(lambda_, num_qubits)

# Adding error to noise model on a list of gates for a single qubit:
noise_model = NoiseModel()
noise_model.add_quantum_error(error, ['id'], qubits = [0]) 

Alternative, adding to a list of gates for all qubits:

In [6]:
# Adding error to noise model on a list of gates for all qubits:

instructions = ['id', 'x', 'y', 'z', 'rx', 'ry', 'rz'] # 'id' is the idle (identity) gate as placeholder.
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, instructions)

# [Amplitude damping channel](https://qiskit.org/ecosystem/aer/stubs/qiskit_aer.noise.amplitude_damping_error.html#qiskit_aer.noise.amplitude_damping_error) (longitudinal relaxation)<a name="amp" /> 

The amplitude damping channel describes the `loss of energy` from a quantum system (the qubit) due to its interaction with an environment. It models the `longitudinal relaxation` from the excited state $|1\rangle$ to the ground state $|0\rangle$ due to `transverse noise` that couples to the qubit in the $x$-$y$ plane [[3]](#). The physical process behind it is called `thermalization` and it is caused by incoherent energy exchange between qubit and environment [[4]](#).

The amplitude damping quantum error channel action on a density matrix (state) $\rho$ is defined by the following CPTP map:

\begin{eqnarray}
\mathcal{E}_{AD}(\rho) = K_0 \rho K_0^\dagger + K_1 \rho K_1^\dagger,
\end{eqnarray}

where $K_0$ and $K_1$ are Kraus operator satisfying $$K_0^\dagger K_0 + K_1^\dagger K_1 = \mathbb{I}.$$

For a 1-qubit (two dimensional) system (using Nielsen & Chuang's representation):

\begin{eqnarray}
K_0 &=& \begin{pmatrix} 1 & 0 \\ 0 & \sqrt{1-\Gamma_1} \end{pmatrix}, \quad
K_1 &=& \begin{pmatrix} 0 & \sqrt{\Gamma_1} \\ 0 & 0 \end{pmatrix}\\
\Gamma_1 &=& \frac{1}{T_1} = 1-e^{-t/T_1}.
\end{eqnarray}

$$$$

Parameters:

- $K_0$: Kraus operator that leaves the $|0\rangle$ state unchanged, but reduces the amplitude of the $|1\rangle$.
- $K_1$: Kraus operator that changes a $|1\rangle$ state to a $|0\rangle$ state. `Corresponds to the physical process of losing a quantum of energy to the environment.`
- $\Gamma_1$: is the amplitude damping error rate parameter. Can be thought of as the probability of losing a photon. 
    - No energy loss ($\Gamma_1=0$).
    - Complete energy loss ($\Gamma_1=1$).
- $T_1$: is the `longitudinal relaxation time` of the process (experimental fitting parameter). It is the time constant to `recover the longitudinal component of the Bloch vector` to its thermal equilibrium [[4]](#).

For a general 1-qubit density matrix 

\begin{eqnarray}
\rho = \begin{pmatrix}\rho_{00}&\rho_{01}\\\rho_{10}&\rho_{11}\end{pmatrix},
\end{eqnarray}

the quantum channel reads
\begin{eqnarray}
\mathcal{E}_{AD}(\rho) = \begin{pmatrix}\rho_{00}+\Gamma_1 \rho_{11} & \sqrt{1-\Gamma_1} \rho_{01} \\ \sqrt{1-\Gamma_1} \rho_{10} & (1-\Gamma_1) \rho_{11}\end{pmatrix}.
\end{eqnarray}

The standard procedure to measure the `longitudinal relaxation time` $T_1$ of a qubit, i.e, the time it takes to decay from the excited state $|1\rangle$ to the ground state $|0\rangle$, is via [inversion recovery](https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware-pulses/calibrating-qubits-pulse.ipynb) [[5]](#). A calibrated drive pulse ($\pi$-pulse) is applied, followed by a time delay $\Delta t$, and a readout pulse to measure the population of $|0\rangle$ as a function of time. This procedure is repeated several times for a different time delay $\Delta t$.

The fitting function to measure $T_1$ reads:

$$f(t) = \alpha_0 + \alpha_1 exp\Big(\frac{-t}{T_1}\Big).$$

Where 

- $t$ is the time delay between the drive and readout pulses [[4]](#). 
- $\alpha_0$ (offset), $\alpha_1$ (amplitude), and $T_1$ are fitting parameters.

# [Phase damping channel](https://qiskit.org/ecosystem/aer/stubs/qiskit_aer.noise.phase_damping_error.html#qiskit_aer.noise.phase_damping_error) (pure dephasing)<a name="pdamp" /> 

The phase damping channel models the `loss of coherence (pure dephasing)` of a quantum system due to its interaction with an environment. "Pure dephasing in the `transverse plane` arises from `longitudinal noise` along the $z$-axis" [[3]](#).

The phase damping quantum error channel action on a density matrix (state) $\rho$ is defined by the following CPTP map:

\begin{eqnarray}
\mathcal{E}_{PD}(\rho) = K_0 \rho K_0^\dagger + K_1 \rho K_1^\dagger,
\end{eqnarray}

where $K_0$ and $K_1$ are Kraus operator satisfying $$K_0^\dagger K_0 + K_1^\dagger K_1 = \mathbb{I}.$$

For a 1-qubit (two dimensional) system (using Nielsen & Chuang's representation):

\begin{eqnarray}
K_0 &=& \begin{pmatrix} 1 & 0 \\ 0 & \sqrt{1-\Gamma_\phi} \end{pmatrix}, \quad
K_1 = \begin{pmatrix} 0 & 0 \\ 0 & \sqrt{\Gamma_\phi} \end{pmatrix}, \\
\Gamma_\phi &=& \frac{1}{T_\phi} = 1-e^{-t/T_\phi}.
\end{eqnarray}

**Recall:** in a density matrix $\rho$, the `diagonal elements` $\rho_{ii} = \langle i|\rho|i\rangle$ `represent the populations` (relative amplitudes) in the different basis states, while the `off-diagonal elements represent the coherences`. 

**Note:** in quantum mechanics, "probability" and "population" are often used interchangeably to refer to the likelihood of finding a quantum system in a particular state.

Parameters:
- $K_0$: Kraus operator that leaves the $|0\rangle$ state unchanged, but reduces the amplitude of the $|1\rangle$.
- $K_1$: Kraus operator that destroys the $|0\rangle$ state and reduces the amplitude of the $|1\rangle$ state.
- $\Gamma_\phi$: is the phase damping error rate parameter. Describes depolarization in the x-y plane of the Bloch sphere [[3]](#).
    - No decoherence loss ($\Gamma_\phi=0$).
    - Complete decoherence loss ($\Gamma_\phi=1$). The off-diagonal elements of $\rho$ are zero.
- $T_\phi$: is the decoherence a.k.a `dephasing time` of the process (experimental fitting parameter).

The fitting function to measure $T_{\phi}$ reads:

$$f(t) = \alpha_0 + \alpha_1 exp\Big(\frac{-t}{T_\phi}\Big).$$

Where 

- $t$ is the time interval between pulses. 
- $\alpha_0$ (offset), $\alpha_1$ (amplitude), and $T_\phi$ are fitting parameters.

# [Phase-amplitude damping error channel](https://qiskit.org/documentation/stable/0.19/stubs/qiskit.providers.aer.noise.phase_amplitude_damping_error.html#phase-amplitude-damping-error) (transverse relaxation)<a name="phaseamplitude" /> 

The phase-amplitude damping channel combines the single-qubit [phase damping](https://qiskit.org/documentation/stable/0.19/stubs/qiskit.providers.aer.noise.phase_damping_error.html#qiskit.providers.aer.noise.phase_damping_error) `(pure dephasing)` and [amplitude damping](https://qiskit.org/documentation/stable/0.19/stubs/qiskit.providers.aer.noise.amplitude_damping_error.html#qiskit.providers.aer.noise.amplitude_damping_error) (`longitudinal energy relaxation)` quantum error channels. This combination leads to `Transverse relaxation` and results in `loss of coherence` at a rate [[3]](#)[[4]](#):

$$\Gamma_2 = \Gamma_1/2 + \Gamma_\phi = \frac{1}{T_2} = \frac{1}{2T_1} + \frac{1}{T_\phi}.$$

The phase-amplitude damping quantum error channel action on a density matrix (state) $\rho$ is defined by the following CPTP map:

$$\mathcal{E}(\rho) = \sum_{i=0}^5 E_i\rho E_i^\dagger.$$

Kraus operators:

\begin{align*}
A_0 &= \sqrt{1-p_1} \begin{pmatrix} 1 & 0 \\ 0 & \sqrt{1-a-b} \end{pmatrix}, \\
A_1 &= \sqrt{1-p_1} \begin{pmatrix} 0 & \sqrt{a} \\ 0 & 0 \end{pmatrix}, \\
A_2 &= \sqrt{1-p_1} \begin{pmatrix} 0 & 0 \\ 0 & \sqrt{b} \end{pmatrix}, \\
B_0 &= \sqrt{p_1} \begin{pmatrix} \sqrt{1-a-b} & 0 \\ 0 & 1 \end{pmatrix}, \\
B_1 &= \sqrt{p_1} \begin{pmatrix} 0 & 0 \\ \sqrt{a} & 0 \end{pmatrix}, \\
B_2 &= \sqrt{p_1} \begin{pmatrix} \sqrt{b} & 0 \\ 0 & 0 \end{pmatrix}.
\end{align*}

With

\begin{align*}
a \equiv \Gamma_1 &= 1 - exp(- t / T_1). \\
b \equiv \Gamma_\phi &= 1 - exp(- t / T_\phi).
\end{align*}

Parameters:

- $\Gamma_2$: is the phase-amplitude damping error rate parameter.
- $a \equiv \Gamma_1$: the amplitude damping error rate parameter.
- $b \equiv \Gamma_\phi$: the phase damping error rate parameter.
- $T_2$: is the `effective decoherence time` of the process (an experimental fitting parameter). It is the `constant decay time of the transverse component` of the Bloch vector to zero [[4]](#).
- $t$: time interval between drive pulses. Also the duration of the interaction between the system and the environment.

The standard procedure to measure the `transverse relaxation time` $T_2$, i.e, the effective coherence time of a qubit, is either a Ramsey or Hahn Echo experiment [[5]](https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware-pulses/calibrating-qubits-pulse.ipynb). 

- **Ramsey**: to observe Ramsey fringes one drives the qubit off-resonance ($\omega_d \ne \omega_q$) with a $X_{\pi/2}$ pulse followed by a $X_{-\pi/2}$ pulse to send the transverse component back to the longitudinal component. Finally, one applies a readout pulse (measurement) at the resonator's frequency $\omega_r$. 
- **Hahn Echo**: the main idea is to apply a calibrated $X_{\pi/2}$ drive pulse at the beginning, followed by a $X_{\pi}$ pulse at time $\tau$ to reverse the phase and another $X_{\pi/2}$ pulse at time $2\tau$. The $X_{\pi}$ pulse at time $\tau$ creates an echo at time $2\tau$. This pulse sequence is then followed by a readout pulse (measurement) of frequency $\omega_r$. 

The fitting function to measure $T_2$ via Ramsey fringes reads [[5]](https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware-pulses/calibrating-qubits-pulse.ipynb):

$$f(t) = \alpha_0 + \alpha_1 cos(\omega_{qd} \tau+ \alpha_2)exp\Big(\frac{-\tau}{T_2}\Big).$$

Where 

- $\tau$ is the time interval between the two $\pi/2$ pulses.
- $\omega_{qd} \equiv \omega_d - \omega_q$ ($\omega_d \ne \omega_q$) is the frequency detuning of the qubit.
- $\omega_{q} \equiv \omega_{01}$ denotes the qubit transition frequency between states $|0\rangle$ and $|1\rangle$.
- $\omega_{d}$ is the pulse (drive) frequency used to drive the qubit.
- $\alpha_0$ (offset), $\alpha_1$ (amplitude), $\alpha_2$ (phase offset) and $T_2$ are fitting parameters.

In [7]:
# Creating a phase-amplitude damping noise model.

# Parameters from real experiment:
t = 0.02                   # (in micro seconds) = 20ns The duration of the interaction between system and environment.
T1 = 10                    # The longitudinal relaxation time (μs) of the qubits.
T2 = 0.25*T1               # The transverse relaxation time (μs) of the qubits.
Tphi = (2*T2*T1)/(2*T1-T2) # The dephasing time (μs) of the qubits.

# Parameters for the error channel:
a = 1 - exp(-t/T1)   # Amplitude damping error (longitudinal relaxation) rate parameter.
b = 1 - exp(-t/Tphi) # Phase damping error (dephasing) rate parameter.
p = 0.01             # The population of the excited state |1> at equilibrium.

# Create phase-amplitude damping error channel:
error = phase_amplitude_damping_error(param_amp = a, param_phase = b, excited_state_population = p)

# Instructions
instructions = ['id', 'x', 'y', 'z', 'rx', 'ry', 'rz'] # 'id' is the idle (identity) gate as placeholder.

# Adding error to noise model on a list of gates for all qubits:
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, instructions)

# [Kraus error channel](https://qiskit.org/documentation/stable/0.19/stubs/qiskit.providers.aer.noise.kraus_error.html#kraus-error)<a name="kraus" /> 

In [8]:
def noise_model_function(T1, T2, t, p = 0.001):
    '''
    Create a 2-qubit noise model with kraus error channel.

    Args:
        - T1 (float): the longitudinal relaxation time (μs) of the qubits.
        - T2 (float): the transverse relaxation time (μs) of the qubits.
        - t (float): time between gate operations.
        - p (float): excited population probability.

    Returns:
        - noise_model (QuantumError): a noise model object with Kraus error added.
    '''
    Tphi = (2*T2*T1)/(2*T1 - T2)
    a = 1 - exp(- t / Tphi)
    
    A1 = [[1,0,0,0], [0,sqrt(1-a),0,0], [0,0,sqrt(1-a),0], [0,0,0,1-a]]
    A2 = [[0,0,0,0], [0,sqrt(a),0,0], [0,0,0,0], [0,0,0,sqrt(1-a)*sqrt(a)]]
    A3 = [[0,0,0,0], [0,0,0,0], [0,0,sqrt(a),0], [0,0,0,sqrt(1-a)*sqrt(a)]]
    A4 = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,a]]

    error = kraus_error([A1,A2,A3,A4])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, ['cx', 'cp'], warnings=False)
    return noise_model

# Params
t = 0.02 # (in micro seconds) = 20ns.
T1 = 10 
T2 = 0.25*T1 

noise_model_function(T1, T2, t)

<NoiseModel on ['cx', 'cp']>

# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/book" width="45" hspace="0px" vspace="0px"></a> References

[1] Nielsen MA, Chuang IL. 2010. Quantum Computation and Quantum Information. New York: [Cambridge Univ. Press.](https://doi.org/10.1017/CBO9780511976667) 10th Anniv. Ed. 
- Chapter 8: Quantum noise and quantum operations.

[2] John Preskill. "Course Information for Physics 219/Computer Science 219 Quantum Computation." [California Institute of Technology.](http://theory.caltech.edu/~preskill/ph229/)

[3] P. Krantz, M. Kjaergaard, F. Yan, T. P. Orlando, S. Gustavsson, W. D. Oliver; A quantum engineer's guide to superconducting qubits. [Applied Physics Reviews](https://doi.org/10.1063/1.5089550) 1 June 2019; 6 (2): 021318. 

[4] Sangil Kwon, Akiyoshi Tomonaga, Gopika Lakshmi Bhai, Simon J. Devitt, Jaw-Shen Tsai; Gate-based superconducting quantum computing. [Journal of Applied Physics](https://doi.org/10.1063/5.0029735) 28 January 2021; 129 (4): 041102. 

[5] [Calibrating qubits using Qiskit Pulse](https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware-pulses/calibrating-qubits-pulse.ipynb).

[6] Qiskit documentation.